# CS-344 Artificial Intelligence - Homework 2 - Uncertainty

# Part 1: Build a spam filter based on Paul Graham’s "A Plan for Spam"

In [16]:
"""
Course: CS 344 - Artificial Intelligence
Instructor: Professor VanderLinden
Name: Joseph Jinn
Date: 3-1-19

Homework 2 - Uncertainty
Spam filter - LISP to Python algorithm conversion (from Paul Graham's A Plan for Spam)

Notes:

Include in your solution, as one test case, the probability tables for the words in the following hard-coded
SPAM/HAM corpus (and only this corpus) using a minimum count threshold of 1 (rather than the 5 used in the algorithm)

Oh, god, functional languages...

Resources Used:

https://stackoverflow.com/questions/2600191/how-can-i-count-the-occurrences-of-a-list-item
(I used this to figure out how to count the number of occurrences of each token.)

https://stackoverflow.com/questions/11068986/how-to-convert-counter-object-to-dict
(convert a "Counter" to a dict

https://stackoverflow.com/questions/3294889/iterating-over-dictionaries-using-for-loops
(how to iterate through all items in a dictionary)

https://stackoverflow.com/questions/3845362/how-can-i-check-if-a-key-exists-in-a-dictionary
(how to check if a key exists or doesn't exist in a dictionary)

https://www.pythoncentral.io/how-to-sort-python-dictionaries-by-key-or-value/
(sort a dictionary by its values)

https://www.tutorialspoint.com/How-to-convert-Python-dictionary-keys-values-to-lowercase
(convert dict keys to lower-case)
"""

############################################################################################
############################################################################################

from collections import Counter
from itertools import islice

############################################################################################

# Spam corpus contains spam e-mails.
spam_corpus = [["I", "am", "spam", "spam", "I", "am"], ["I", "do", "not", "like", "that", "spamiam"]]
# Ham corpus contains non-spam e-mails.
ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]

"""
The especially observant will notice that while I consider each corpus to be a single long stream of text for purposes 
of counting occurrences, I use the number of emails in each, rather than their combined length, 
as the divisor in calculating spam probabilities. This adds another slight bias to protect against false positives.
"""
# Number of spam and non-spam e-mails messages (not words).
number_bad_message = len(spam_corpus)
number_good_messages = len(spam_corpus)

# Threshold value for the spam filter algorithm.
algorithm_threshold_value = 1
spam_message_threshold_value = 0.9
interesting_tokens_threshold_value = 15


############################################################################################
############################################################################################


class SpamFilter:
    """
    The SpamFilter class accepts a corpus and analyzes it to determine what words are spam and what words are
    legitimate.

    # TODO - refactor spam filter to be class-based rather than just individual floating functions.
    Note: Placeholder for now.
    """

    def __init__(self, corpus, threshold):
        """
        Class constructor.

        :param corpus: list of words to analyze.
        :param threshold:  threshold value for the algorithm.
        """
        self.corpus = corpus
        self.threshold = threshold


############################################################################################
############################################################################################

def word_occurrences(corpus):
    """
    Counts the number of times each word occurs in the message.

    :param corpus: list of words to count occurrences for.
    :return:  dictionary of each word and their occurrences.

    Here's a sketch of how I do statistical filtering.
    I start with one corpus of spam and one of nonspam mail.
    At the moment each one has about 4000 messages in it.
    I scan the entire text, including headers and embedded html and javascript, of each message in each corpus.
    I currently consider alphanumeric characters, dashes, apostrophes, and dollar signs to be part of tokens,
    and everything else to be a token separator. (There is probably room for improvement here.)
    I ignore tokens that are all digits, and I also ignore html comments,
    not even considering them as token separators.

    I count the number of times each token (ignoring case, currently) occurs in each corpus.
    At this stage I end up with two large hash tables, one for each corpus, mapping tokens to number of occurrences.
    """
    occur_array = []

    for e in corpus:
        occur = Counter(e)
        occur_array.append(occur)

    return occur_array


############################################################################################

def individual_word_spam_chance(spam_words_dict, non_spam_words_dict, threshold):
    """
    TODO - ensure this is how the algorithm is meant to function - ask Professor VanderLinden.
    Determines the spam'liness of each individual word in the message.

    :param spam_words_dict: dictionary containing spam words.
    :param non_spam_words_dict: dictionary containing non-spam words.
    :param threshold: threshold value for statistical algorithm.
    :return:  spam'liness value of each individual word as a dictionary.

        Next I create a third hash table, this time mapping each token to the probability that an email containing
    it is a spam, which I calculate as follows [1]:

    (let ((g (* 2 (or (gethash word good) 0)))
          (b (or (gethash word bad) 0)))
       (unless (< (+ g b) 5)
         (max .01
              (min .99 (float (/ (min 1 (/ b nbad))
                                 (+ (min 1 (/ g ngood))
                                    (min 1 (/ b nbad)))))))))

    where word is the token whose probability we're calculating,
    good and bad are the hash tables I created in the first step,
    and ngood and nbad are the number of nonspam and spam messages respectively.

    FIXME - is this the right way of calculating the probabilities? (refer to passage below from "A Plan for Spam")

    One question that arises in practice is what probability to assign to a word you've never seen, i.e.
    one that doesn't occur in the hash table of word probabilities.
    I've found, again by trial and error, that .4 is a good number to use.
    If you've never seen a word before, it is probably fairly innocent; spam words tend to be all too familiar.
    """

    words_spam_chance = {}

    # Iterate through all non-spam words.
    for good_key, good_value in non_spam_words_dict.items():

        # If no associated value for each word set to 0, otherwise multiply value by 2.
        if good_value is None:
            good_occurrences = 0
        else:
            good_occurrences = 2 * good_value

        # Determine if non-spam word has match in spam word dictionary.
        # If no, set to 0.  If yes, set to value of non-spam word.
        if good_key not in spam_words_dict:
            bad_occurrences = 0
        else:
            bad_occurrences = spam_words_dict[good_key]

        # Statistical algorithm to calculate the associated probability for each word.
        if good_occurrences + bad_occurrences > threshold:
            probability = max(0.01, min(0.99, min(1.0, bad_occurrences / number_bad_message) /
                                        min(1.0, good_occurrences / number_good_messages) +
                                        min(1.0, bad_occurrences / number_bad_message)))
        else:
            probability = 0

        # Store to dictionary each word and their associated probability.
        words_spam_chance[good_key] = probability

    # Return our dictionary of stored words spam probabilities.
    return words_spam_chance


############################################################################################

def message_spam_chance(word_probabilities_dict):
    """
    Determines the final probability of being spam.

    :param word_probabilities_dict: individual probabilities for each word.
    :return: probability the message is spam.

    If probs is a list of the fifteen individual probabilities, you calculate the combined probability thus:

    (let ((prod (apply #'* probs)))
      (/ prod (+ prod (apply #'* (mapcar #'(lambda (x)
                                             (- 1 x))
                                         probs)))))
    """

    # Remove the keys from the dictionary and store only the associated values.
    word_probability_values = sorted(word_probabilities_dict.values())
    print("\nword spam probability values only, keys removed: " + str(word_probability_values))

    # Calculate the product of all individual word probabilities.
    product_of_probabilities = 1.0
    for each_probability in word_probability_values:
        product_of_probabilities *= each_probability

    print("product of individual values: " + str(product_of_probabilities))

    # Determine the complement of all individual word probabilities.
    word_probability_complement_values = []
    for each_probability in word_probability_values:
        complement = 1.00 - each_probability
        word_probability_complement_values.append(complement)

    print("word spam complement probability values: " + str(word_probability_complement_values))

    # Calculate the product of all complement probabilities.
    product_of_complement_probabilities = 1.0
    for each_complement_probability in word_probability_complement_values:
        product_of_complement_probabilities *= each_complement_probability

    print("product of complement values: " + str(product_of_complement_probabilities))

    spam_message_probability = product_of_probabilities / \
                               (product_of_probabilities + product_of_complement_probabilities)

    print("final probability message is spam: " + str(spam_message_probability))

    return spam_message_probability


############################################################################################


def find_interesting_tokens(word_spam_chance_dict):
    """
    Prunes dictionary containing the words in the message to the most interesting 15 tokens based
    on the size of their deviation from the "Neutral" value of 0.5

    :param word_spam_chance_dict: dictionary containing the spam probabilities of each word.
    :return: the 15 most interesting words and their associated spam probabilities.

    FIXME - set cut-off value to 15 after we are finished testing (interesting_tokens_threshold_value).

    When new mail arrives, it is scanned into tokens, and the most interesting fifteen tokens,
    where interesting is measured by how far their spam probability is from a neutral .5,
    are used to calculate the probability that the mail is spam.
    """
    # If more than 15 tokens, prune to the most "interesting" 15.
    if len(word_spam_chance_dict) > interesting_tokens_threshold_value:

        # Determine the 15 tokens with the largest deviation from neutral 0.5.
        normalized_word_spam_chance = {}
        for key, value in word_spam_chance_dict.items():
            normalized_word_spam_chance[key] = abs(0.5 - value)
        print("\nnormalized word spam chances: " + str(normalized_word_spam_chance))

        # Sort dictionary so that largest deviations are at the front.
        sorted_dict = sorted(normalized_word_spam_chance.items())
        print("\nmy sorted dict with normalized values: " + str(sorted_dict))

        # Slice dictionary so only first 15 key-value pairs are left.
        slice_dict = islice(sorted_dict, interesting_tokens_threshold_value)

        # Convert to dictionary as islice returns an iterator.
        first15 = {}
        for each in slice_dict:
            first15[each[0]] = each[1]
        print("\nfirst 15 tokens with normalized keys and values: " + str(first15))

        # Un-normalize and return to original values by assigning original values.
        first15_unnormalized = {}
        for key, value in first15.items():
            first15_unnormalized[key] = word_spam_chance_dict[key]
        print("first 15 tokens un-normalized keys and values: " + str(first15_unnormalized))
    else:
        return word_spam_chance_dict

    return first15_unnormalized


def convert_to_lowercase_and_combine_dict(spam_occurrences, nonspam_occurrences):
    """
    Convert all keys to lower-case as words should not be case-sensitive.
    Also, collapse all messages and their individual words into one combined dictionary containing
    all unique component words and their occurrences.

    :param spam_occurrences: contains all spam words and their occurrences as multiple dictionaries.
    :param nonspam_occurrences: contains all non-spam words and their occurrences as multiple dictionaries.
    :return:  two dictionaries, one for spam and one for non-spam, containing their lower-case words and
    associated occurrences.
    """
    lowercase_only_spam_word_occurrences = {}
    for each_dictionary in spam_occurrences:
        for key, value in each_dictionary.items():
            if key.lower() not in lowercase_only_spam_word_occurrences:
                lowercase_only_spam_word_occurrences[key.lower()] = value
            else:
                lowercase_only_spam_word_occurrences[key.lower()] += value
    print("\nlower-case only word spam word occurrences: " + str(lowercase_only_spam_word_occurrences))

    lowercase_only_nonspam_word_occurrences = {}
    for each_dictionary in nonspam_occurrences:
        for key, value in each_dictionary.items():
            if key.lower() not in lowercase_only_nonspam_word_occurrences:
                lowercase_only_nonspam_word_occurrences[key.lower()] = value
            else:
                lowercase_only_nonspam_word_occurrences[key.lower()] += value
    print("lower-case only word non-spam word occurrences: " + str(lowercase_only_nonspam_word_occurrences))

    all_occurrences = [lowercase_only_spam_word_occurrences, lowercase_only_nonspam_word_occurrences]
    return all_occurrences


############################################################################################
############################################################################################


if __name__ == '__main__':
    """
    Pithy Introduction.
    Executes the program.
    """
    print("\nExecuting spam filter algorithm!")
    print("I like Spam! - Delicious!")
    print("\n\n")

    # Get occurrences of each word in the list of words - returned as dictionary.
    spamWordOccurrencesDict = word_occurrences(spam_corpus)
    nonSpamWordOccurrencesDict = word_occurrences(ham_corpus)
    print("\noccurrences of each word in spam dictionary: " + str(spamWordOccurrencesDict))
    print("occurrences of each word in non-spam dictionary: " + str(nonSpamWordOccurrencesDict))

    # Convert words to lower-case, sum occurrences of unique words, and return as list containing 2 dictionaries -
    # one for spam and another for non-spam.
    combined_occurrences = convert_to_lowercase_and_combine_dict(spamWordOccurrencesDict, nonSpamWordOccurrencesDict)

    # Determine probability that each word in the message is spam - returned as dictionary.
    word_spam_chance = individual_word_spam_chance(combined_occurrences[0],
                                                   combined_occurrences[1], algorithm_threshold_value)
    print("words spam probabilities: " + str(word_spam_chance))

    # Obtain the 15 most interesting tokens in the message.
    interesting_words_only = find_interesting_tokens(word_spam_chance)

    # Obtain the spam message probability value.
    result = message_spam_chance(interesting_words_only)

    # Compare spam message probability value against threshold spam probability value.
    if result >= spam_message_threshold_value:
        print("Spam!!!!")
    else:
        print("Not spam.")

############################################################################################
############################################################################################



Executing spam filter algorithm!
I like Spam! - Delicious!




occurrences of each word in spam dictionary: [Counter({'I': 2, 'am': 2, 'spam': 2}), Counter({'I': 1, 'do': 1, 'not': 1, 'like': 1, 'that': 1, 'spamiam': 1})]
occurrences of each word in non-spam dictionary: [Counter({'do': 1, 'i': 1, 'like': 1, 'green': 1, 'eggs': 1, 'and': 1, 'ham': 1}), Counter({'i': 1, 'do': 1})]

lower-case only word spam word occurrences: {'i': 3, 'am': 2, 'spam': 2, 'do': 1, 'not': 1, 'like': 1, 'that': 1, 'spamiam': 1}
lower-case only word non-spam word occurrences: {'do': 2, 'i': 2, 'like': 1, 'green': 1, 'eggs': 1, 'and': 1, 'ham': 1}
words spam probabilities: {'do': 0.99, 'i': 0.99, 'like': 0.99, 'green': 0.01, 'eggs': 0.01, 'and': 0.01, 'ham': 0.01}

word spam probability values only, keys removed: [0.01, 0.01, 0.01, 0.01, 0.99, 0.99, 0.99]
product of individual values: 9.702990000000001e-09
word spam complement probability values: [0.99, 0.99, 0.99, 0.99, 0.010000000000000009, 0.01000000000000

# Question: Graham argues that this is a Bayesian approach to SPAM. What makes it Bayesian?

Unlike feature-recognizing filters which assigns an arbitrary spam "score" to an e-mail, the Bayaesian approach assigns actual probabilities.  Therefore, you know what you are measuring and there is little ambiguity about how the evidence should be combined to calculate the chances of the e-mail being spam.

The Bayesian approach considers all evidence, the good evidence decreasing the likelihood of the message being spam and the bad evidence increasing the likelihood of the message being spam.



# Part 2: Bayesian networks

# Bayesian network used:

In [17]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://cs.calvin.edu/courses/cs/344/kvlinden/05bayesnets/images/figure14_12.png")

In [18]:
'''
This module implements the Bayesian network shown in the text, Figure 14.2.
It's taken from the AIMA Python code.

@author: kvlinden
@version Jan 2, 2013
'''

from probability import BayesNet, enumeration_ask, elimination_ask, gibbs_ask, rejection_sampling, likelihood_weighting

# Utility variables
T, F = True, False

# From AIMA code (probability.py) - Fig. 14.2 - burglary example
cloudy = BayesNet([
    ('Cloudy', '', 0.50),
    ('Rain', 'Cloudy', {T: 0.80, F: 0.20}),
    ('Sprinkler', 'Cloudy', {T: 0.10, F: 0.50}),
    ('WetGrass', 'Sprinkler Rain', {(T, T): 0.99, (T, F): 0.90, (F, T): 0.90, (F, F): 0.00})
    ])

# Compute P(Cloudy)
print("\nP(Cloudy)")
print(enumeration_ask('Cloudy', dict(), cloudy).show_approx())
# elimination_ask() is a dynamic programming version of enumeration_ask().
print(elimination_ask('Cloudy', dict(), cloudy).show_approx())
# gibbs_ask() is an approximation algorithm helps Bayesian Networks scale up.
print(gibbs_ask('Cloudy', dict(), cloudy).show_approx())
# See the explanation of the algorithms in AIMA Section 14.4.
print(rejection_sampling('Cloudy', dict(), cloudy).show_approx())
print(likelihood_weighting('Cloudy', dict(), cloudy).show_approx())

###################################################################################################

# Compute P(Sprinkler | cloudy)
print("\nP(Sprinkler | cloudy)")
print(enumeration_ask('Sprinkler', dict(Cloudy=T), cloudy).show_approx())
# elimination_ask() is a dynamic programming version of enumeration_ask().
print(elimination_ask('Sprinkler', dict(Cloudy=T), cloudy).show_approx())
# gibbs_ask() is an approximation algorithm helps Bayesian Networks scale up.
print(gibbs_ask('Sprinkler', dict(Cloudy=T), cloudy).show_approx())
# See the explanation of the algorithms in AIMA Section 14.4.
print(rejection_sampling('Sprinkler', dict(Cloudy=T), cloudy).show_approx())
print(likelihood_weighting('Sprinkler', dict(Cloudy=T), cloudy).show_approx())

###################################################################################################

# Compute P(Cloudy| the sprinkler is running and it’s not raining)
print("\nP(Cloudy| the sprinkler is running and it’s not raining)")
print(enumeration_ask('Cloudy', dict(Sprinkler=T, Rain=F), cloudy).show_approx())
# elimination_ask() is a dynamic programming version of enumeration_ask().
print(elimination_ask('Cloudy', dict(Sprinkler=T, Rain=F), cloudy).show_approx())
# gibbs_ask() is an approximation algorithm helps Bayesian Networks scale up.
print(gibbs_ask('Cloudy', dict(Sprinkler=T, Rain=F), cloudy).show_approx())
# See the explanation of the algorithms in AIMA Section 14.4.
print(rejection_sampling('Cloudy', dict(Sprinkler=T, Rain=F), cloudy).show_approx())
print(likelihood_weighting('Cloudy', dict(Sprinkler=T, Rain=F), cloudy).show_approx())

###################################################################################################

# Compute P(WetGrass | it’s cloudy, the sprinkler is running and it’s raining)
print("\nP(WetGrass | it’s cloudy, the sprinkler is running and it’s raining)")
print(enumeration_ask('WetGrass', dict(Cloudy=T, Sprinkler=T, Rain=T), cloudy).show_approx())
# elimination_ask() is a dynamic programming version of enumeration_ask().
print(elimination_ask('WetGrass', dict(Cloudy=T, Sprinkler=T, Rain=T), cloudy).show_approx())
# gibbs_ask() is an approximation algorithm helps Bayesian Networks scale up.
print(gibbs_ask('WetGrass', dict(Cloudy=T, Sprinkler=T, Rain=T), cloudy).show_approx())
# See the explanation of the algorithms in AIMA Section 14.4.
print(rejection_sampling('WetGrass', dict(Cloudy=T, Sprinkler=T, Rain=T), cloudy).show_approx())
print(likelihood_weighting('WetGrass', dict(Cloudy=T, Sprinkler=T, Rain=T), cloudy).show_approx())

###################################################################################################

# Compute P(Cloudy | the grass is not wet)
print("\nP(Cloudy | the grass is not wet)")
print(enumeration_ask('Cloudy', dict(WetGrass=F), cloudy).show_approx())
# elimination_ask() is a dynamic programming version of enumeration_ask().
print(elimination_ask('Cloudy', dict(WetGrass=F), cloudy).show_approx())
# gibbs_ask() is an approximation algorithm helps Bayesian Networks scale up.
print(gibbs_ask('Cloudy', dict(WetGrass=F), cloudy).show_approx())
# See the explanation of the algorithms in AIMA Section 14.4.
print(rejection_sampling('Cloudy', dict(WetGrass=F), cloudy).show_approx())
print(likelihood_weighting('Cloudy', dict(WetGrass=F), cloudy).show_approx())

# TODO - fix module not found error in jupyter notebook.


ModuleNotFoundError: No module named 'probability'

# Hand calculated probabilities for Bayesian network:

Note: Refer to screen captures in Homework 2 directory and/or turned in hard-copy of hand calculations. (will enter hand-calculations using latex syntax if/when I have time)

<img style="float:center; transform: rotate(90deg); margin: 0 10px 10px 0" src="bayesian_network_hand_calc_page1.jpg" />

<img style="float:center; transform: rotate(90deg); margin: 0 10px 10px 0" src="bayesian_network_hand_calc_page2.jpg" />

<img style="float:center; transform: rotate(90deg); margin: 0 10px 10px 0" src="bayesian_network_hand_calc_page3.jpg" />

Note: TODO - Images are cut-off for some reason.  Fix it.

**Example *causal* computation** 

This computes the probability that John calls given that there as been a burglary but no earthquake.

$\begin{aligned}
    \textbf{P}(J | b \land \neg e)
        &= \alpha \sum_a{\textbf{P}(J, a, b, \neg e)} \\
        &= \alpha \sum_a{\textbf{P}(J|a) \cdot P(a|b,\neg e) \cdot P(b) \cdot P(\neg e)} \\
        &= \alpha \cdot P(b) \cdot P(\neg e) \cdot \sum_a{P(J|a) \cdot P(a|b,-e)} \\
        &= \alpha \cdot 0.001 \cdot 0.998 \cdot \langle(0.9 \cdot 0.94 + 0.05 \cdot 0.06), (0.1 \cdot 0.94 + 0.95 \cdot 0.06)\rangle \\
        &= \alpha \langle0.000847, 0.000150698\rangle \\
        &= \langle0.85, 0.15\rangle
    \end{aligned}$


**Example *diagnostic* computation** 

This computes the probability that there as been a burglary given that the alarm has rung.

$\begin{aligned}
                \textbf{P}(B | a)
                    &= \alpha \sum_e{\textbf{P}(B, e, a)} \\
                    &= \alpha \sum_e{\textbf{P}(B) * P(e) * P(a | B, e)} \\
                    &= \alpha * \langle (0.001 * 0.002 * 0.95 + 0.001 * 0.998 * 0.94), (0.999 * 0.002 * 0.29 + 0.999 * 0.998 * 0.001)\rangle \\
                    &= \alpha * \langle0.00094002, 0.001576422\rangle \\
                    &= \langle0.374, 0.626\rangle
                \end{aligned}$


# Question: Compute the number of independent values in the full joint probability distribution for this domain. Assume that no conditional independence relations are known to hold between these values.

Note: Ensure my answer is correct.

There are n=4 binary random variables, therefore the number of independent parameters is 2^(n=4)-1.  This results in 2^4-1 = 15 independent parameters in total.

Resources Used:

http://www.cs.brandeis.edu/~cs134/K_F_Ch3.pdf


# Compute the number of independent values in the Bayesian network for this domain. Assume the conditional independence relations implied by the Bayes network.

Note: Ensure my answer is correct.

There are n=4 binary random variables, therefore the number of independent parameters is 2*(n=4)-1.  This results in 2*4-1 = 7 independent parameters in total.

Resources Used:

http://www.cs.brandeis.edu/~cs134/K_F_Ch3.pdf